In [1]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 141.7 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21581 sha256=befdbe36a348e1161eddb93c02a0cb6d62a2bca98b8ecf68cb1161ce3025ad9a
  Stored in directory: /home/datalore/.cache/pip/wheels/90/74/b1/9b54c896b8d9409e9268329d4d45ede8a8040abe91c8879932
Successfully built promise


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

2025-05-13 11:20:15.169445: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

#wczytanie danych
(train_ds,test_ds),info = tfds.load(
    "cifar10",
    split=['train','test'],
    as_supervised=True,
    with_info=True
)

Dataset cifar10 downloaded and prepared to /home/datalore/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [5]:
for image,labels in train_ds.take(1):
    print(f"Images shape: {image.shape}")
    print(f"Labels shape: {labels.shape}")

Images shape: (32, 32, 3)
Labels shape: ()


2025-05-13 11:25:06.905327: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-05-13 11:25:06.913227: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-05-13 11:25:06.914309: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
#normalizacja danych
def preprocess(image,label):
    image = tf.cast(image,tf.float32)/255.0
    return image,label

In [7]:
BATCH_SIZE=128

In [8]:
train_ds = (
    train_ds
    .shuffle(10000)
    .map(preprocess,num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_ds = (
    test_ds
    .map(preprocess,num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [9]:
for image,labels in train_ds.take(1):
    print(f"Images shape: {image.shape}")
    print(f"Labels shape: {labels.shape}")

Images shape: (128, 32, 32, 3)
Labels shape: (128,)


2025-05-13 11:30:08.380152: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-05-13 11:30:08.392058: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
#prosta sieć neuronowa
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,32,3)),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

/opt/python/envs/default_3_11/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [12]:
#trenujemy model
model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.2758 - loss: 2.0075 - val_accuracy: 0.3549 - val_loss: 1.7943
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.3822 - loss: 1.7297 - val_accuracy: 0.3944 - val_loss: 1.6951
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.4180 - loss: 1.6366 - val_accuracy: 0.4425 - val_loss: 1.5693
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.4413 - loss: 1.5731 - val_accuracy: 0.4468 - val_loss: 1.5482
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.4511 - loss: 1.5396 - val_accuracy: 0.4775 - val_loss: 1.4933
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.4758 - loss: 1.4862 - val_accuracy: 0.4760 - val_loss: 1.4873
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.4856 - loss: 1.4551 - val_accuracy: 0.4821 - val_loss: 1.4573
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.4914 - loss: 1.4295 - val_ac